In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
state = AppState()
config = {}
trading_historical_data = {}
account = None
account = TradingAccount(app)
state.last_action = "SELL"
dataframe_dict = []
pd.set_option('display.max_columns', None)

def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])
    rsi = df_last['rsi14'].values[0]

    action = '' 
    
    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'rri_buy: {rri_buy}')

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    if len(df) == 0:
        return df
    return df.iloc[iterations - 1:iterations]

def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [2]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    ta = TechnicalAnalysis(trading_dataCopy)
    ta.addAll()
    df = ta.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            # technical indicators
            ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
            ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
            goldencross = bool(df_last['goldencross'].values[0])
            macdgtsignal = bool(df_last['macdgtsignal'].values[0])
            macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
            ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
            ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
            macdltsignal = bool(df_last['macdltsignal'].values[0])
            macdltsignalco = bool(df_last['macdltsignalco'].values[0])
            obv = float(df_last['obv'].values[0])
            obv_pc = float(df_last['obv_pc'].values[0])
            elder_ray_buy = bool(df_last['eri_buy'].values[0])
            elder_ray_sell = bool(df_last['eri_sell'].values[0])
            rri = df_last['rri'].values[0]
            last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
            last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
            last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
            last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
            rri_buy = df_last['rri_buy'].values[0]
            rri_sell = df_last['rri_sell'].values[0]
            rsi = df_last['rsi14'].values[0]
            rsi_last = df_last['rsi_last'].values[0]
            rsi_prev = df_last['rsi_prev'].values[0]
            
            hammer = bool(df_last['hammer'].values[0])
            inverted_hammer = bool(df_last['inverted_hammer'].values[0])
            hanging_man = bool(df_last['hanging_man'].values[0])
            shooting_star = bool(df_last['shooting_star'].values[0])
            three_white_soldiers = bool(df_last['three_white_soldiers'].values[0])
            three_black_crows = bool(df_last['three_black_crows'].values[0])
            morning_star = bool(df_last['morning_star'].values[0])
            evening_star = bool(df_last['evening_star'].values[0])
            three_line_strike = bool(df_last['three_line_strike'].values[0])
            abandoned_baby = bool(df_last['abandoned_baby'].values[0])
            morning_doji_star = bool(df_last['morning_doji_star'].values[0])
            evening_doji_star = bool(df_last['evening_doji_star'].values[0])
            two_black_gapping = bool(df_last['two_black_gapping'].values[0])
            
            # if simulation interations < 200 set goldencross to true
            if app.isSimulation() and state.iterations < 200:
                goldencross = True
            buy_score = 0
            if ema12gtema26: buy_score += 3
            if ema12gtema26co: buy_score += 3
            if macdgtsignal: buy_score += 3
            if goldencross: buy_score += 2
            if macdgtsignalco: buy_score +=3
            if obv_pc > -5: buy_score += 3
            if elder_ray_buy: buy_score += 2
            if not elder_ray_sell: buy_score += 2
            if rri_buy and (ema12gtema26 or ema12gtema26co) and (ema12gtema26co or macdgtsignalco):
                buy_score += 10
            elif rri_buy and (ema12gtema26 and macdgtsignal):
                buy_score += 8
            elif rri_buy and (ema12gtema26 or macdgtsignal):
                buy_score += 6
            elif rri_buy:
                buy_score += 4
            candle_stick_score = 0
            # weak up
            if hammer is True: candle_stick_score += 1
            if inverted_hammer is True: candle_stick_score += 1
            # weak down
            if shooting_star is True: candle_stick_score -= 1
            if hanging_man is True: candle_stick_score -= 1
            # strong up
            if three_white_soldiers is True: candle_stick_score += 2
            if morning_star is True: candle_stick_score += 2
            # strong down
            if three_black_crows is True: candle_stick_score -= 2
            if evening_star is True: candle_stick_score -= 2
            # reliable up
            if three_line_strike is True: candle_stick_score += 3
            if abandoned_baby is True: candle_stick_score += 3
            if morning_doji_star is True: candle_stick_score += 3
            # reliable down
            if evening_doji_star is True: candle_stick_score -= 3
            if two_black_gapping is True: candle_stick_score -= 3
                
            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append({"action": state.action,
                                       "coin_pair": coin_pair,
                                       "timestamp": df_last['date'][0].isoformat(),
                                       "buy_score": buy_score, 
                                       "candle_stick_score":candle_stick_score,
                                       "granularity": app.getGranularity(),
                                       "price": price,
                                       "one_hr_high": one_hr_high, 
                                       "one_hr_low": one_hr_low, 
                                       "three_hr_high": three_hr_high, 
                                       "three_hr_low": three_hr_low, 
                                       "three_six_high": three_six_high,
                                       "three_six_low": three_six_low, 
                                       "six_12_high": six_12_high, 
                                       "six_12_low": six_12_low, 
                                       "six_hr_high": six_hr_high, 
                                       "six_hr_low": six_hr_low,
                                       "nine_hr_high": nine_hr_high, 
                                       "nine_hr_low": nine_hr_low, 
                                       "twelve_hr_high": twelve_hr_high, 
                                       "twelve_hr_low": twelve_hr_low,
                                       "ema12gtema26": ema12gtema26,
                                       "ema12gtema26co": ema12gtema26co,
                                       "goldencross": goldencross,
                                       "macdgtsignal": macdgtsignal, 
                                       "macdgtsignalco": macdgtsignalco, 
                                       "obv": obv, 
                                       "obv_pc": obv_pc, 
                                       "elder_ray_buy": elder_ray_buy,
                                       "elder_ray_sell": elder_ray_sell, 
                                       "rri": rri, 
                                       "rri_buy": rri_buy, 
                                       "rri_sell": rri_sell, 
                                       "last_3_rri_buy": last_3_rri_buy, 
                                       "last_5_rri_buy": last_5_rri_buy, 
                                       "last_3_rri_sell": last_3_rri_sell, 
                                       "last_5_rri_sell": last_5_rri_sell, 
                                       "rsi": rsi,
                                       "rsi_last": rsi_last,
                                       "rsi_prev": rsi_prev,})
    

In [3]:
%%time
app = PyCryptoBot()
state = AppState()
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
                          'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
                          'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
                          'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
                          'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
                          'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
                          'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
                          'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
                          'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
                          'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
                          'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
                          'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
                          'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
                          'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
                       'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
                       'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
                       'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
                       'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
                       'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
                       'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
                       'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
                       'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
                       'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
                       'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
                       'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
                       'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
                       'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
                       'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
                       'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
                       'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
                       'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
                       'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
                       'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
                       'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
                       'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
                       'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
                       'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
                       'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
                        'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
                       'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
                       'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
                        'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
                       'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
                       'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
                       'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
                       'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
                       'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
                       'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
                       'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
                       'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
                       'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
                       'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
                       'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
                       'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
                       'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
                       'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
                       'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
                       'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
                       'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
                       'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
                       'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
                       'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
                       'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
                       'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
                       'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
                       'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
                       'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
                       'MDXBUSD', 'MASKBUSD']
# binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB', 'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT','NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',]
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict)


calculating  WTCBNB
calculating  BATBNB
calculating  NEOBNB
calculating  IOTABNB
calculating  XLMBNB
calculating  WABIBNB
calculating  LTCBNB
calculating  WAVESBNB
calculating  ICXBNB
calculating  BLZBNB
calculating  ZILBNB
calculating  ONTBNB
calculating  WANBNB
calculating  ADABNB
calculating  ZENBNB
calculating  EOSBNB
calculating  THETABNB
calculating  XRPBNB
calculating  ENJBNB
calculating  TRXBNB
calculating  ETCBNB
calculating  SCBNB
calculating  MFTBNB
calculating  VETBNB
calculating  RVNBNB
calculating  MITHBNB
calculating  BTTBNB
calculating  HOTBNB
calculating  FETBNB
calculating  XMRBNB
calculating  ZECBNB
calculating  IOSTBNB
calculating  CELRBNB
calculating  DASHBNB
calculating  MATICBNB
calculating  ATOMBNB
calculating  ONEBNB
calculating  FTMBNB
calculating  ALGOBNB
calculating  ANKRBNB
calculating  WINBNB
calculating  COSBNB
calculating  COCOSBNB
calculating  PERLBNB
calculating  CHZBNB
calculating  BANDBNB
calculating  XTZBNB
calculating  HBARBNB
calculating  STXBNB
c

calculating  ADABUSD
calculating  BCHBUSD
calculating  QTUMBUSD
calculating  VETBUSD
calculating  ICXBUSD
calculating  BNTBUSD
calculating  ATOMBUSD
calculating  DASHBUSD
calculating  NEOBUSD
calculating  WAVESBUSD
calculating  XTZBUSD
calculating  BATBUSD
calculating  ENJBUSD
calculating  NANOBUSD
calculating  ONTBUSD
calculating  RVNBUSD
calculating  ALGOBUSD
calculating  BTTBUSD
calculating  TOMOBUSD
calculating  XMRBUSD
calculating  ZECBUSD
calculating  DATABUSD
calculating  SOLBUSD
calculating  CTSIBUSD
calculating  HBARBUSD
calculating  MATICBUSD
calculating  WRXBUSD
calculating  ZILBUSD
calculating  KNCBUSD
calculating  LRCBUSD
calculating  IQBUSD
calculating  GBPBUSD
calculating  DGBBUSD
calculating  COMPBUSD
calculating  SXPBUSD
calculating  SNXBUSD
calculating  MKRBUSD
calculating  RUNEBUSD
calculating  MANABUSD
calculating  DOGEBUSD
calculating  ZRXBUSD
calculating  FIOBUSD
calculating  AVABUSD
calculating  IOTABUSD
calculating  BALBUSD
calculating  YFIBUSD
calculating  SRMB

In [4]:
buy_dataframe.describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev
count,15739.000000,15739.000000,15739.0,15739.000000,15727.000000,15727.000000,15727.000000,15727.000000,15625.000000,15625.000000,15353.000000,15353.000000,15727.000000,15727.000000,15727.000000,15727.000000,15727.000000,15727.000000,1.573900e+04,15739.000000,15730.000000,15739.000000,15739.000000,15525.000000
mean,12.519156,-0.002986,300.0,311.285688,1.346187,-1.253539,2.266032,-2.199430,3.153884,-3.335814,4.366635,-4.692702,3.297296,-3.169929,4.091096,-3.920444,4.761363,-4.536287,-2.039984e+06,28.650145,-0.545615,47.159375,50.619543,48.985023
std,3.857440,0.345703,0.0,3205.191562,2.073108,1.372414,3.216956,2.140854,6.309327,4.035359,10.133377,5.652686,4.656512,2.932666,6.098929,3.540725,8.345922,4.073129,9.626074e+08,3524.188790,0.595513,11.382989,10.227455,10.157830
min,4.000000,-5.000000,300.0,0.000100,-1.810000,-24.010000,-1.630000,-24.640000,-12.690000,-78.180000,-14.600000,-78.740000,-1.070000,-24.640000,-1.020000,-78.180000,-1.020000,-78.740000,-4.281282e+10,-80252.610000,-1.000000,10.359490,11.864593,10.359490
25%,9.000000,0.000000,300.0,0.061560,0.410000,-1.780000,0.680000,-3.020000,0.240000,-5.310000,0.210000,-7.710000,1.000000,-4.410000,1.200000,-5.570000,1.350000,-6.460000,-1.162138e+05,-8.470000,-0.898365,39.322415,44.743094,43.242999
50%,13.000000,0.000000,300.0,0.779800,0.930000,-0.930000,1.490000,-1.640000,2.150000,-2.550000,2.870000,-3.770000,2.210000,-2.250000,2.670000,-2.870000,3.050000,-3.330000,6.436610e+03,-0.490000,-0.812500,48.220450,50.980782,50.000000
75%,16.000000,0.000000,300.0,6.385500,1.720000,-0.350000,2.840000,-0.720000,4.570000,-0.830000,6.330000,-0.990000,4.100000,-1.090000,4.920000,-1.380000,5.570000,-1.590000,9.736940e+05,7.800000,-0.750000,55.389394,56.913967,55.057189
max,26.000000,5.000000,300.0,46622.440000,140.260000,2.110000,140.260000,1.750000,185.200000,39.760000,397.840000,51.990000,140.260000,1.720000,185.200000,1.720000,397.840000,1.720000,1.862116e+10,340144.820000,1.000000,89.262641,94.701557,89.605991


In [5]:
buy_dataframe.to_csv("buy_df_5m_ema_rri_rules.csv")

### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour

